# HIMALAYAS

### In brief...

### For every iteration, the df is inspected and then filtered by its pubDate (if there's one). Finally, it creates a new df that will be transformed into a new table in the local postgreSQL DB.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import mysql.connector
import psycopg2
import numpy as np
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse

In [2]:
df = pd.read_csv('/Users/juanreyesgarcia/Library/CloudStorage/OneDrive-FundacionUniversidaddelasAmericasPuebla/DEVELOPER/PROJECTS/CRAWLER_ALL/OUTPUTS/himalaya.csv')

##### Play with the settings...

In [3]:
pd.set_option('display.max_colwidth', 150)

In [4]:
pd.set_option("display.max_rows", None)

# EDA Checklist 

In [5]:
df[:100]

,title,link,pubdate,location,description
0,['Director of Software Engineering'],https://himalayas.app/companies/smartasset/jobs/director-of-software-engineering,1 hour ago,United States only,Software Engineer
1,['Strategic Finance Associate'],https://himalayas.app/companies/ramp/jobs/strategic-finance-associate,1 hour ago,United States only,Finance Associate
2,['Revenue Analyst Jr'],https://himalayas.app/companies/ilia/jobs/revenue-analyst-jr-5069396623,1 hour ago,United States only,Revenue Analyst
3,"['Director, Enterprise Product Growth']",https://himalayas.app/companies/upwork/jobs/director-enterprise-product-growth,3 hours ago,United States only,Director of Growth
4,['Lead Product Marketing Manager | Segmentation &amp; Solutions'],https://himalayas.app/companies/ramp/jobs/lead-product-marketing-manager-segmentation-solutions,4 hours ago,United States only,Product Marketing Manager
5,['Senior Program Manager Finance Systems'],https://himalayas.app/companies/upwork/jobs/senior-program-manager-finance-systems,4 hours ago,United States only,Finance Manager
6,['Learning Analytics Intern'],https://himalayas.app/companies/newsela/jobs/learning-analytics-intern,5 hours ago,United States only,Data Scientist
7,"['Director of Engineering, Mobile &amp; Collaboration']",https://himalayas.app/companies/upwork/jobs/director-of-engineering-mobile-collaboration,5 hours ago,United States only,Director of Engineering
8,"['Director, Sales Finance']",https://himalayas.app/companies/upwork/jobs/director-sales-finance,5 hours ago,APAC only,Director of Finance
9,['Director of Sales - APAC'],https://himalayas.app/companies/hotjar/jobs/director-of-sales-apac-2660332286,5 hours ago,United States only,Director of Sales


In [6]:
df.tail()

,title,link,pubdate,location,description
35,['Account Executive'],https://himalayas.app/companies/karbon/jobs/account-executive-2241909699,1 day ago,NaN,Customer Support
36,['Customer Support Associate - Accounts &amp; Billing'],https://himalayas.app/companies/webflow/jobs/customer-support-associate-accounts-billing,1 day ago,NaN,Business Analyst
37,['Principal Business Strategy Analyst'],https://himalayas.app/companies/smartasset/jobs/principal-business-strategy-analyst,1 day ago,NaN,Back End Developer
38,"['Staff Backend Engineer, Flow Builder Team']",https://himalayas.app/companies/postscript/jobs/staff-backend-engineer-flow-builder-team,1 day ago,NaN,Director of Business Intelligence
39,['Director of Business Intelligence and Data Engineering'],https://himalayas.app/companies/recharge/jobs/director-of-business-intelligence-and-data-engineering,1 day ago,NaN,NaN


### Describe the DF (current)

In [7]:
df.describe()

,title,link,pubdate,location,description
count,40,40,40,31,39
unique,40,40,17,7,34
top,['Director of Software Engineering'],https://himalayas.app/companies/smartasset/jobs/director-of-software-engineering,1 day ago,United States only,Director of Product
freq,1,1,9,24,3


### Checking for missing values DF (current)

In [8]:
df.isnull().sum()

title          0
link           0
pubdate        0
location       9
description    1
dtype: int64

### Fill missing values with "NaN" DF (current)

In [9]:
# Fill missing values with "NaN"
df.fillna("NaN", inplace=True)

In [10]:
df.isnull().sum()

title          0
link           0
pubdate        0
location       0
description    0
dtype: int64

### Checking for duplicates DF (current)

In [11]:
df.duplicated().sum()

0

In [12]:
df.describe()

,title,link,pubdate,location,description
count,40,40,40,40,40
unique,40,40,17,8,35
top,['Director of Software Engineering'],https://himalayas.app/companies/smartasset/jobs/director-of-software-engineering,1 day ago,United States only,Director of Product
freq,1,1,9,24,3


In [13]:
df.dtypes

title          object
link           object
pubdate        object
location       object
description    object
dtype: object

### Mask to convert %minutes|hours to 1 day ago so it can be parse to date time (cba to say it was posted today, maybe later should be fixed)

In [14]:
mask = df['pubdate'].str.contains('minutes|hours')

In [15]:
df.loc[mask, 'pubdate'] = "1 day ago"

In [16]:
df.head()

,title,link,pubdate,location,description
0,['Director of Software Engineering'],https://himalayas.app/companies/smartasset/jobs/director-of-software-engineering,1 hour ago,United States only,Software Engineer
1,['Strategic Finance Associate'],https://himalayas.app/companies/ramp/jobs/strategic-finance-associate,1 hour ago,United States only,Finance Associate
2,['Revenue Analyst Jr'],https://himalayas.app/companies/ilia/jobs/revenue-analyst-jr-5069396623,1 hour ago,United States only,Revenue Analyst
3,"['Director, Enterprise Product Growth']",https://himalayas.app/companies/upwork/jobs/director-enterprise-product-growth,1 day ago,United States only,Director of Growth
4,['Lead Product Marketing Manager | Segmentation &amp; Solutions'],https://himalayas.app/companies/ramp/jobs/lead-product-marketing-manager-segmentation-solutions,1 day ago,United States only,Product Marketing Manager


# From relative date strings to date time...

In [17]:
# create a mask to identify rows with relative date strings
mask = df['pubdate'].str.contains('ago')

# apply the relative date calculation to the date_string column
df.loc[mask, 'datetime'] = pd.Timestamp.today() - df.loc[mask, 'pubdate'].apply(lambda x: relativedelta(days=int(x.split()[0])))

# convert the datetime column to datetime format
df['datetime'] = pd.to_datetime(df['datetime'], infer_datetime_format=True)


/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_95761/3229561357.py:5: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[mask, 'datetime'] = pd.Timestamp.today() - df.loc[mask, 'pubdate'].apply(lambda x: relativedelta(days=int(x.split()[0])))


In [18]:
df.head(50)

,title,link,pubdate,location,description,datetime
0,['Director of Software Engineering'],https://himalayas.app/companies/smartasset/jobs/director-of-software-engineering,1 hour ago,United States only,Software Engineer,2023-03-07 23:52:08.593140
1,['Strategic Finance Associate'],https://himalayas.app/companies/ramp/jobs/strategic-finance-associate,1 hour ago,United States only,Finance Associate,2023-03-07 23:52:08.593140
2,['Revenue Analyst Jr'],https://himalayas.app/companies/ilia/jobs/revenue-analyst-jr-5069396623,1 hour ago,United States only,Revenue Analyst,2023-03-07 23:52:08.593140
3,"['Director, Enterprise Product Growth']",https://himalayas.app/companies/upwork/jobs/director-enterprise-product-growth,1 day ago,United States only,Director of Growth,2023-03-07 23:52:08.593140
4,['Lead Product Marketing Manager | Segmentation &amp; Solutions'],https://himalayas.app/companies/ramp/jobs/lead-product-marketing-manager-segmentation-solutions,1 day ago,United States only,Product Marketing Manager,2023-03-07 23:52:08.593140
5,['Senior Program Manager Finance Systems'],https://himalayas.app/companies/upwork/jobs/senior-program-manager-finance-systems,1 day ago,United States only,Finance Manager,2023-03-07 23:52:08.593140
6,['Learning Analytics Intern'],https://himalayas.app/companies/newsela/jobs/learning-analytics-intern,1 day ago,United States only,Data Scientist,2023-03-07 23:52:08.593140
7,"['Director of Engineering, Mobile &amp; Collaboration']",https://himalayas.app/companies/upwork/jobs/director-of-engineering-mobile-collaboration,1 day ago,United States only,Director of Engineering,2023-03-07 23:52:08.593140
8,"['Director, Sales Finance']",https://himalayas.app/companies/upwork/jobs/director-sales-finance,1 day ago,APAC only,Director of Finance,2023-03-07 23:52:08.593140
9,['Director of Sales - APAC'],https://himalayas.app/companies/hotjar/jobs/director-of-sales-apac-2660332286,1 day ago,United States only,Director of Sales,2023-03-07 23:52:08.593140


### Drop pubDate, change name of datetime & reindex...

In [19]:
#Drop it
df = df.drop('pubdate', axis=1)

In [20]:
#Change name of datetime to pubDate 
df = df.rename(columns={'datetime': 'pubdate'})

In [21]:
#reindex
df = df.reindex(columns=['title', 'link', 'description', 'pubdate', 'location'])

In [22]:
df.head()

,title,link,description,pubdate,location
0,['Director of Software Engineering'],https://himalayas.app/companies/smartasset/jobs/director-of-software-engineering,Software Engineer,2023-03-07 23:52:08.593140,United States only
1,['Strategic Finance Associate'],https://himalayas.app/companies/ramp/jobs/strategic-finance-associate,Finance Associate,2023-03-07 23:52:08.593140,United States only
2,['Revenue Analyst Jr'],https://himalayas.app/companies/ilia/jobs/revenue-analyst-jr-5069396623,Revenue Analyst,2023-03-07 23:52:08.593140,United States only
3,"['Director, Enterprise Product Growth']",https://himalayas.app/companies/upwork/jobs/director-enterprise-product-growth,Director of Growth,2023-03-07 23:52:08.593140,United States only
4,['Lead Product Marketing Manager | Segmentation &amp; Solutions'],https://himalayas.app/companies/ramp/jobs/lead-product-marketing-manager-segmentation-solutions,Product Marketing Manager,2023-03-07 23:52:08.593140,United States only


## Filter rows by a date range (this reduces the number of rows... duh)

In [23]:
# filter rows by a date range
start_date = pd.to_datetime('2016-01-01')
end_date = pd.to_datetime('2023-02-15')

#for df
date_range_filter = (df['pubdate'] >= start_date) & (df['pubdate'] <= end_date)
df = df.loc[~date_range_filter]

In [24]:
df = df.sort_values(by='pubdate')

In [25]:
df.describe(datetime_is_numeric=True)

,pubdate
count,40
mean,2023-03-07 16:04:08.593139968
min,2023-02-22 23:52:08.593140
25%,2023-03-07 23:52:08.593139968
50%,2023-03-07 23:52:08.593139968
75%,2023-03-07 23:52:08.593139968
max,2023-03-07 23:52:08.593140


In [26]:
df.describe()

/var/folders/3x/_059946j4lvcjpvshx_dj5f40000gn/T/ipykernel_95761/3627053830.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  df.describe()


,title,link,description,pubdate,location
count,40,40,40,40,40
unique,40,40,35,2,8
top,['Senior User Experience Designer'],https://himalayas.app/companies/power-home-remodeling/jobs/senior-user-experience-designer,Director of Product,2023-03-07 23:52:08.593140,United States only
freq,1,1,3,39,24
first,NaN,NaN,NaN,2023-02-22 23:52:08.593140,NaN
last,NaN,NaN,NaN,2023-03-07 23:52:08.593140,NaN


In [27]:
df.dtypes

title                  object
link                   object
description            object
pubdate        datetime64[ns]
location               object
dtype: object

### Make a copy just in case

In [28]:
df1 = df.copy()

In [29]:
# replace NaT values in the DataFrame with None
df = df.replace({np.nan: None, pd.NaT: None})

In [30]:
df

,title,link,description,pubdate,location
26,['Senior User Experience Designer'],https://himalayas.app/companies/power-home-remodeling/jobs/senior-user-experience-designer,Account Executive,2023-02-22 23:52:08.593140,United States only
0,['Director of Software Engineering'],https://himalayas.app/companies/smartasset/jobs/director-of-software-engineering,Software Engineer,2023-03-07 23:52:08.593140,United States only
22,['Strategic Alliances Manager'],https://himalayas.app/companies/pagerduty/jobs/strategic-alliances-manager,Partnerships Manager,2023-03-07 23:52:08.593140,United States only
23,"['Director of Product Management, Tax']",https://himalayas.app/companies/upwork/jobs/director-of-product-management-tax,Director of Product,2023-03-07 23:52:08.593140,United States only
24,['Global Finance Director'],https://himalayas.app/companies/ilia/jobs/global-finance-director,Finance Director,2023-03-07 23:52:08.593140,US &amp; CA
25,['Sales Engineer'],https://himalayas.app/companies/wirewheel/jobs/sales-engineer-6285211537,Sales Engineer,2023-03-07 23:52:08.593140,United States only
27,['Strategic Account Executive - remote UK'],https://himalayas.app/companies/gitlab/jobs/strategic-account-executive-remote-uk,Business Development,2023-03-07 23:52:08.593140,United States only
28,['Business Development Manager - Product &amp; Platform Partnerships'],https://himalayas.app/companies/upwork/jobs/business-development-manager-product-platform-partnerships,Sales Development Representative,2023-03-07 23:52:08.593140,United States only
21,"['Assessment Contributor, Spanish Language']",https://himalayas.app/companies/newsela/jobs/assessment-contributor-spanish-language,Content Creator,2023-03-07 23:52:08.593140,United States only
29,['Sales Development Representative'],https://himalayas.app/companies/newsela/jobs/sales-development-representative,Growth Marketer,2023-03-07 23:52:08.593140,North America only


# PostgreSQL

### This code creates a new table per iteration

In [31]:
# create a connection to the PostgreSQL database
cnx = psycopg2.connect(user='postgres', password='3312', host='localhost', port='5432', database='postgres')

# create a cursor object
cursor = cnx.cursor()

# get the name of the next table to create
get_table_name_query = '''
    SELECT COUNT(*) FROM information_schema.tables
    WHERE table_name LIKE 'himalayas_%'
'''
cursor.execute(get_table_name_query)
result = cursor.fetchone()
next_table_number = result[0] + 1
next_table_name = 'himalayas_{}'.format(next_table_number)

# prepare the SQL query to create a new table
create_table_query = '''
    CREATE TABLE {} (
        title VARCHAR(255),
        link VARCHAR(255),
        description VARCHAR(1000),
        pubdate TIMESTAMP,
        location VARCHAR(255)
    )
'''.format(next_table_name)

# execute the create table query
cursor.execute(create_table_query)

# insert the DataFrame into the PostgreSQL database as a new table
for index, row in df.iterrows():
    insert_query = '''
        INSERT INTO {} (title, link, description, pubdate, location)
        VALUES (%s, %s, %s, %s, %s)
    '''.format(next_table_name)
    values = (row['title'], row['link'], row['description'], row['pubdate'], row['location'])
    cursor.execute(insert_query, values)

# commit the changes to the database
cnx.commit()

# close the cursor and connection
cursor.close()
cnx.close()